# Custom Skill for Form Recognizer Service

### Constants

In [1]:
# Azure Storage account settings:
azure_storage_account = 'YOUR_STORAGE_ACCOUNT'
blob_connection_string = "YOUR_STORAGE_CONNECTION_STRING"
blob_container = "margies-forms"
train_container = "margies-train"
local_folder = 'insurance_forms'

# Azure Search constants
azsearch_url = "YOUR_SEARCH_SERVICE.search.windows.net"
azsearch_key = "YOUR_AZURE_SEARCH_KEY"

# Cognitive Services Key
cog_svc_key = "YOUR_COGNITIVE_SERVICES_KEY"

# Form recognizer
form_recognizer_key = "YOUR_FORM_RECOGNIZER_KEY"

In [3]:
# Download and extract document files
! rm -R insurance_forms
! wget https://aka.ms/kmoh-forms -O insurance_forms.zip
! unzip -o -d insurance_forms insurance_forms.zip

--2019-08-13 20:45:29--  https://kmohstore.blob.core.windows.net/kmohdata/PDF_Forms.zip
Resolving webproxy (webproxy)... 10.36.4.1
Connecting to webproxy (webproxy)|10.36.4.1|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 840309 (821K) [application/x-zip-compressed]
Saving to: ‘PDF_Forms.zip’

PDF_Forms.zip       100%[===================>] 820.61K  1.10MB/s    in 0.7s    

2019-08-13 20:45:30 (1.10 MB/s) - ‘PDF_Forms.zip’ saved [840309/840309]

Archive:  PDF_Forms.zip
  inflating: insurance_forms/Insurance Form 01.pdf  
  inflating: insurance_forms/Insurance Form 02.pdf  
  inflating: insurance_forms/Insurance Form 03.pdf  
  inflating: insurance_forms/Insurance Form 04.pdf  
  inflating: insurance_forms/Insurance Form 05.pdf  
  inflating: insurance_forms/Insurance Form 06.pdf  
  inflating: insurance_forms/Insurance Form 07.pdf  
  inflating: insurance_forms/Insurance Form 08.pdf  
  inflating: insurance_forms/Insurance Form 09.pdf  


In [4]:
# install the Azure Storage SDK
!pip install azure-storage

     |████████████████████████████████| 194kB 2.3MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
# Create containers
from azure.storage.blob import BlockBlobService, ContainerPermissions

block_blob_service = BlockBlobService(connection_string = blob_connection_string)

try:
    block_blob_service.create_container(blob_container)
    block_blob_service.create_container(train_container)
except Exception as ex:
    print(ex)

In [6]:
# Upload form images to storage

import os
doc_no = 0
try:
    filelist = [file_name for file_name in sorted(os.listdir(local_folder))]
    for file_name in filelist:
        # Upload all the forms to the internal container
        print('Uploading ', file_name)
        file_path = os.path.join(local_folder, file_name)
        block_blob_service.create_blob_from_path(blob_container, file_name, file_path)
        # Upload only the first 6 forms to the training container
        doc_no += 1
        if doc_no < 7:
            print('Uploading ', file_name)
            file_path = os.path.join(local_folder, file_name)
            block_blob_service.create_blob_from_path(train_container, file_name, file_path)
    print('done!')
except Exception as ex:
    print(ex)

Uploading  Insurance Form 01.pdf
Uploading  Insurance Form 01.pdf
Uploading  Insurance Form 02.pdf
Uploading  Insurance Form 02.pdf
Uploading  Insurance Form 03.pdf
Uploading  Insurance Form 03.pdf
Uploading  Insurance Form 04.pdf
Uploading  Insurance Form 04.pdf
Uploading  Insurance Form 05.pdf
Uploading  Insurance Form 05.pdf
Uploading  Insurance Form 06.pdf
Uploading  Insurance Form 06.pdf
Uploading  Insurance Form 07.pdf
Uploading  Insurance Form 08.pdf
Uploading  Insurance Form 09.pdf
done!


In [7]:

import datetime
# Train a Form Recognizer model
from requests import post as http_post
import json

# Endpoint URL
form_recognizer_url = "https://westus2.api.cognitive.microsoft.com/formrecognizer/v1.0-preview/custom"

# Use SAS to access training forms
sas_key = block_blob_service.generate_container_shared_access_signature(train_container, ContainerPermissions.READ + ContainerPermissions.LIST, datetime.datetime.utcnow() + datetime.timedelta(hours=1))
source = "https://" + block_blob_service.account_name + ".blob.core.windows.net/" + train_container + "/?" + sas_key
headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': form_recognizer_key,
}

# Call the training API
url = form_recognizer_url + "/train" 
body = {"source": source}
try:
    print("Training...")
    resp = http_post(url = url, json = body, headers = headers)
    print("Response status code: %d" % resp.status_code)
    print(json.dumps(resp.json(), sort_keys=True, indent=2))
    model_id = resp.json()["modelId"]
    print(model_id)
except Exception as e:
    print(str(e))

Training...
Response status code: 200
{
  "errors": [],
  "modelId": "4ac4e4a3-386b-4c9a-af45-3f4b01887d3b",
  "trainingDocuments": [
    {
      "documentName": "Insurance Form 01.pdf",
      "errors": [],
      "pages": 1,
      "status": "success"
    },
    {
      "documentName": "Insurance Form 02.pdf",
      "errors": [],
      "pages": 1,
      "status": "success"
    },
    {
      "documentName": "Insurance Form 03.pdf",
      "errors": [],
      "pages": 1,
      "status": "success"
    },
    {
      "documentName": "Insurance Form 04.pdf",
      "errors": [],
      "pages": 1,
      "status": "success"
    },
    {
      "documentName": "Insurance Form 05.pdf",
      "errors": [],
      "pages": 1,
      "status": "success"
    },
    {
      "documentName": "Insurance Form 06.pdf",
      "errors": [],
      "pages": 1,
      "status": "success"
    }
  ]
}
4ac4e4a3-386b-4c9a-af45-3f4b01887d3b


In [9]:
# Test the model
from azure.storage.blob import BlockBlobService, BlobPermissions
from requests import post as http_post
import json
import os
import requests
from datetime import datetime, timedelta

# Endpoint URL
form_recognizer_url = "https://westus2.api.cognitive.microsoft.com/formrecognizer/v1.0-preview/custom"

# Get the URL and an SAS token for the first form
file_name = os.listdir(os.path.join(local_folder))[0]
sas_token = block_blob_service.generate_blob_shared_access_signature(container_name=blob_container, blob_name=file_name, expiry=datetime.utcnow() + timedelta(hours=1), permission=BlobPermissions.READ)
sas_url = "https://" + block_blob_service.account_name + ".blob.core.windows.net/" + blob_container + "/" + file_name.replace(" ", "%20") + "?" + sas_token

# Get the form data
image_bytes = requests.get(sas_url).content

# Use the Form Recognizer model to read the form
headers = {
    # Request headers
    'Content-Type': 'application/pdf',
    'Ocp-Apim-Subscription-Key': form_recognizer_key,
}

try:
    url = form_recognizer_url + "/models/" + model_id + "/analyze" 
    resp = http_post(url = url, data = image_bytes, headers = headers)
    # parse the response
    print("Response status code: %d" % resp.status_code)
    for page in resp.json()["pages"]:
        for pair in page["keyValuePairs"]:
            print(pair["key"][0]["text"], pair["value"][0]["text"])
except Exception as e:
    print(str(e))

Response status code: 200
Name: Dayaakar Nandamuri
Address: 321 Big Street
City: Bellevue
Postal Code: 980321
Country: United States
Date of Birth: 07/01/1988
Destination City: Dubai
Destination Country: UAE
Departure Date: 05/05/2018
Return Date: 05/12/2018
Signature: D Nandamuri
Date: 05/01/2018


### Create an Azure Function that Wraps the Form Recognizer Model

Create a Python Azure Function with the following code in the **__init__.py** code file:

```
import logging
import os
import sys
import json
import requests
from requests import post as http_post
import azure.functions as func


def main(req: func.HttpRequest) -> func.HttpResponse:

    # The result will be a "values" bag
    result = {
        "values": []
    }
    statuscode = 200

    try:
        values = req.get_json().get('values')
        logging.info(values)

        for rec in values:
            # Construct the basic JSON response for this record
            val = {
                    "recordId": rec['recordId'],
                    "data": {
                    },
                    "errors": None,
                    "warnings": None
                }
            try:
                # get the image to be processed from the input record
                image_url = rec['data']['url']
                sas_token = rec['data']['sas_token']
                sas_url = image_url + sas_token
                image_response = requests.get(sas_url)
                image_content = image_response.content

                # Parse Form
                form_recognizer_url = "https://westus2.api.cognitive.microsoft.com/formrecognizer/v1.0-preview/custom"
                form_recognizer_key = "<YOUR_FORM_RECOGNIZER_KEY>"
                model_id="<YOUR_MODEL_ID>"

                headers = {
                    # Request headers
                    'Content-Type': 'application/pdf',
                    'Ocp-Apim-Subscription-Key': form_recognizer_key,
                }

                url = form_recognizer_url + "/models/" + model_id + "/analyze" 
                resp = http_post(url = url, data = image_content, headers = headers)
                if resp.status_code == 200:
                    for page in resp.json()["pages"]:
                        for pair in page["keyValuePairs"]:
                            #print(pair["key"][0]["text"], pair["value"][0]["text"])
                            field_name = pair["key"][0]["text"].lower().replace(" ", "")
                            field_value = None
                            if (len(pair["value"]) > 0):
                                field_value = pair["value"][0]["text"]
                            val["data"][field_name] = field_value
            except Exception as ex:
                logging.info(ex)
                # An error occured for this text record, so add lists of errors and warning
                val["errors"] =[{"message": "An error occurred processing the content."}]
                val["warnings"] = [{"message": "One or more inputs failed to process."}]
            finally:
                # Add the value for this record to the response
                result["values"].append(val)
    except Exception as ex:
        statuscode = 500
        logging.info(ex)
        # A global error occurred, so return an error response
        val = {
                "recordId": None,
                "data": {
                    "text":None
                },
                "errors": [{"message": ex.args}],
                "warnings": [{"message": "The request failed to process."}]
            }
        result["values"].append(val)
    finally:
        # Return the response
        return func.HttpResponse(body=json.dumps(result), mimetype="application/json", status_code=statuscode)

```

Modify the **requirements.txt** file to include the *requests* library as follows:

```
requests
```

In [30]:
# Test the function

from requests import post as http_post
import json
import base64

# URL for the Azure Function
func = 'https://margiesfuncs.azurewebsites.net/api/form_func'

# Get the URL and an SAS token for the first form
file_name = os.listdir(os.path.join(local_folder))[0]
url = "https://" + block_blob_service.account_name + ".blob.core.windows.net/" + blob_container + "/" + file_name.replace(" ", "%20")
sas_token = "?" + block_blob_service.generate_blob_shared_access_signature(container_name=blob_container, blob_name=file_name, expiry=datetime.utcnow() + timedelta(hours=1), permission=BlobPermissions.READ)


# Send a request in the same format as the Azure Search service custom skill

body = {
        "values": [
            {
                "recordId": "a1",
                "data":
                {
                    "url":url,
                    "sas_token": sas_token
                }
            }
       ]
    }

headers = {
    # Request headers
    'Content-Type': 'application/json'
}

print("Request")
print(json.dumps(body, indent=2))

resp = http_post(url = func, data = json.dumps(body), headers = headers)
print(resp)
print(resp)
print("Response status code: %d" % resp.status_code)
print(json.dumps(resp.json(), indent=2))

Request
{
  "values": [
    {
      "recordId": "a1",
      "data": {
        "url": "https://margiesstore.blob.core.windows.net/margies-forms/Insurance%20Form%2007.pdf",
        "sas_token": "?se=2019-08-13T23%3A29%3A59Z&sp=r&sv=2017-04-17&sr=b&sig=bVcwk4KQy07HiZChgt27mj0Ek34u6MPlWeBl2SI13Mc%3D"
      }
    }
  ]
}
<Response [200]>
<Response [200]>
Response status code: 200
{
  "values": [
    {
      "recordId": "a1",
      "data": {
        "name:": "Dayaakar Nandamuri",
        "address:": "321 Big Street",
        "city:": "Bellevue",
        "postalcode:": "980321",
        "country:": "United States",
        "dateofbirth:": "07/01/1988",
        "destinationcity:": "Dubai",
        "destinationcountry:": "UAE",
        "departuredate:": "05/05/2018",
        "returndate:": "05/12/2018",
        "signature:": "D Nandamuri",
        "date:": "05/01/2018"
      },
      "errors": null,
      "warnings": null
    }
  ]
}


### Create an Index with a Custom Skill for Form Recognition

In [37]:
## Helper function for calling Azure Search API

def azsearch_rest(request_type="GET", function_name="servicestats", body=None):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }

    # Request parameters
    params = urllib.parse.urlencode({
        'api-version':'2019-05-06-Preview'
    })
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/{0}?{1}".format(function_name, params)
        conn.request(request_type, request_path, body, headers)
        response = conn.getresponse()
        print(response.status)
        data = response.read().decode("UTF-8")
        result = None
        if len(data) > 0:
            result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API")

Ready to use the REST API


### Create a Data Source

In [38]:
import json

# Define the request body
body = {   
    "name" : "internal-forms",  
    "description" : "forms.",  
    "type" : "azureblob",
    "credentials" :
    { "connectionString" : blob_connection_string
    },  
    "container" : { "name" : blob_container }
} 

try:
    result = azsearch_rest(request_type="POST", function_name="datasources", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
except Exception as ex:
    print(ex)

201
{
  "@odata.context": "https://margies-search.search.windows.net/$metadata#datasources/$entity",
  "@odata.etag": "\"0x8D7203EB3A0075C\"",
  "container": {
    "name": "margies-forms",
    "query": null
  },
  "credentials": {
    "connectionString": "DefaultEndpointsProtocol=https;AccountName=margiesstore;AccountKey=jx2f1r0z7xgyb5Z54636gCtZNv11JswBqUEvypu90CBFLcnzqrqcK8v46gUoplPc/JFGj0qyfwYIx5ufPolLWg==;EndpointSuffix=core.windows.net"
  },
  "dataChangeDetectionPolicy": null,
  "dataDeletionDetectionPolicy": null,
  "description": "forms.",
  "name": "internal-forms",
  "subtype": null,
  "type": "azureblob"
}


### Create an Index

In [39]:
# Create an index

import json

# Define the request body
body = {
  "fields": [
    {
      # Unique ID for each document
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "searchable": True,
      "filterable": True,
      "facetable": False,
      "sortable": True
    },
    {
     # The file path of the document
      "name": "url",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The file name of the document
      "name": "file_name",
      "type": "Edm.String",
      "sortable": True,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # The size of the document
      "name": "size",
      "type": "Edm.Int64",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The last-modified date of the document
      "name": "last_modified",
      "type": "Edm.DateTimeOffset",
      "sortable": True,
      "searchable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # The content in the document
      "name": "content",
      "type": "Edm.String",
      "sortable": False,
      "searchable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # name
      "name": "name",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # address
      "name": "address",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # address
      "name": "city",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # address
      "name": "post_code",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # address
      "name": "country",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # address
      "name": "birth_date",
      "type": "Edm.String",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # destinationcity
      "name": "destination_city",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # destinationcity
      "name": "destination_country",
      "type": "Edm.String",
      "searchable": True,
      "sortable": False,
      "filterable": True,
      "facetable": False
    },
    {
      # destinationcity
      "name": "departure_date",
      "type": "Edm.String",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # destinationcity
      "name": "return_date",
      "type": "Edm.String",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    },
    {
      # destinationcity
      "name": "policy_date",
      "type": "Edm.String",
      "searchable": False,
      "sortable": True,
      "filterable": True,
      "facetable": False
    }
  ]
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexes/form-recognizer-index", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)
    

201
{
  "@odata.context": "https://margies-search.search.windows.net/$metadata#indexes/$entity",
  "@odata.etag": "\"0x8D7203EB8B8D699\"",
  "analyzers": [],
  "charFilters": [],
  "corsOptions": null,
  "defaultScoringProfile": null,
  "encryptionKey": null,
  "fields": [
    {
      "analyzer": null,
      "facetable": false,
      "filterable": true,
      "indexAnalyzer": null,
      "key": true,
      "name": "id",
      "retrievable": true,
      "searchAnalyzer": null,
      "searchable": true,
      "sortable": true,
      "synonymMaps": [],
      "type": "Edm.String"
    },
    {
      "analyzer": null,
      "facetable": false,
      "filterable": true,
      "indexAnalyzer": null,
      "key": false,
      "name": "url",
      "retrievable": true,
      "searchAnalyzer": null,
      "searchable": true,
      "sortable": true,
      "synonymMaps": [],
      "type": "Edm.String"
    },
    {
      "analyzer": null,
      "facetable": false,
      "filterable": true,
      "ind

### Create a Skillset with the Custom Skill

In [52]:
# Define the request body
body = {
  "description": "Extract and enhance text",
  "skills":
  [
      # Use a custom skill to find the most common words in each document
          {
            "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
            "description": "custom skill",
            "uri": func,
            "batchSize":1,
            "context": "/document",
            "inputs": [
              {
                "name": "url",
                "source": "/document/url"
              },
              {
                "name": "sas_token",
                "source": "/document/metadata_storage_sas_token"
              }
            ],
            "outputs": [
              {
                "name": "name:",
                "targetName": "name"
              },
              {
                "name": "address:",
                "targetName": "address"
              },
              {
                "name": "city:",
                "targetName": "city"
              },
              {
                "name": "postalcode:",
                "targetName": "post_code"
              },
              {
                "name": "country:",
                "targetName": "country"
              },
              {
                "name": "dateofbirth:",
                "targetName": "birth_date"
              },
              {
                "name": "destinationcity:",
                "targetName": "destination_city"
              },
              {
                "name": "destinationcountry:",
                "targetName": "destination_country"
              },
              {
                "name": "departuredate:",
                "targetName": "departure_date"
              },
              {
                "name": "returndate:",
                "targetName": "return_date"
              },
              {
                "name": "date:",
                "targetName": "policy_date"
              }
            ]
          }
      ],
      "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "margies-cog-svc",
        "key": cog_svc_key
      }
    }  

try:
    result = azsearch_rest(request_type="PUT", function_name="/skillsets/form-recognizer-skillset", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


### Create an Indexer

In [53]:
import json

body = {
  "name":"form-recognizer-indexer", 
  "dataSourceName" : "internal-forms",
  "skillsetName" : "form-recognizer-skillset",
  "targetIndexName" : "form-recognizer-index",
  "fieldMappings" : [
      # Map the soure content fields to index fields
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "id",
          "mappingFunction" : 
            { "name" : "base64Encode" }
        },
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "url"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "file_name"
        },
        {
          "sourceFieldName" : "content",
          "targetFieldName" : "content"
        },
        {
          "sourceFieldName" : "metadata_storage_size",
          "targetFieldName" : "size"
        },
        {
          "sourceFieldName" : "metadata_storage_last_modified",
          "targetFieldName" : "last_modified"
        }
   ],
  "outputFieldMappings" : 
  [
      # Map the output from custom skill to index fields
        {
          "sourceFieldName" : "/document/name", 
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "/document/address", 
          "targetFieldName" : "address"
        },
        {
          "sourceFieldName" : "/document/city", 
          "targetFieldName" : "city"
        },
        {
          "sourceFieldName" : "/document/post_code", 
          "targetFieldName" : "post_code"
        },
        {
          "sourceFieldName" : "/document/country", 
          "targetFieldName" : "country"
        },
        {
          "sourceFieldName" : "/document/birth_date", 
          "targetFieldName" : "birth_date"
        },
        {
          "sourceFieldName" : "/document/destination_city", 
          "targetFieldName" : "destination_city"
        },
        {
          "sourceFieldName" : "/document/destination_country", 
          "targetFieldName" : "destination_country"
        },
        {
          "sourceFieldName" : "/document/departure_date", 
          "targetFieldName" : "departure_date"
        },
        {
          "sourceFieldName" : "/document/return_date", 
          "targetFieldName" : "return_date"
        },
        {
          "sourceFieldName" : "/document/policy_date", 
          "targetFieldName" : "policy_date"
        }
  ],
  "parameters":
  {
    "maxFailedItems":-1,
    "maxFailedItemsPerBatch":-1,
    "configuration": 
    {
        "dataToExtract": "contentAndMetadata",
        "imageAction": "generateNormalizedImages"
    }
  }
}  

try:
    result = azsearch_rest(request_type="PUT", function_name="indexers/form-recognizer-indexer", body=json.dumps(body))
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))

except Exception as e:
    print('Error:')
    print(e)

204


In [54]:
import time

try:
    # Reset the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/form-recognizer-indexer/reset", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
        
    # re-run the indexer.
    result = azsearch_rest(request_type="POST", function_name="/indexers/form-recognizer-indexer/run", body=None)
    if result != None:
        print(json.dumps(result, sort_keys=True, indent=2))
    


except Exception as e:
    print('Error:')
    print(e)

204
202


In [55]:
# Get Indexer status until complete

import time, json

try:
    complete = False
    while (complete == False):
        result = azsearch_rest(request_type="GET", function_name="indexers/form-recognizer-indexer/status")
        state = result["status"]
        if result['lastResult'] is not None:
            state = result['lastResult']['status']
        print (state)
        if state in ("success", "error"):
            complete = True
        time.sleep(1)

except Exception as e:
    print('Error:')
    print(e)

200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
inProgress
200
success


### Search the Index

In [56]:
def azsearch_query(index, params):
    # Imports and constants
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

    # Request headers.
    headers = {
        'Content-Type': 'application/json',
        'api-key': azsearch_key
    }
    
    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(azsearch_url)
        request_path = "/indexes/{0}/docs?{1}".format(index, params)
        conn.request("GET", request_path, None, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")
        result = json.loads(data)
        return result

    except Exception as ex:
        raise ex

print("Ready to use the REST API for Queries")

Ready to use the REST API for Queries


In [57]:
import urllib.parse, json

search_terms = input("Search?\n")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, name, address, city, post_code, birth_date, destination_city, destination_country, departure_date, return_date, policy_date',
    'api-version':'2019-05-06-Preview'
})

try:
    result = azsearch_query(index="form-recognizer-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    
    for value in result["value"]:
        print(value["name"], "from", value["city"], "travelling to", value["destination_city"], "on", value["departure_date"])
except Exception as e:
    print('Error:')
    print(e)

Search?
London
Hits: 2
Josefina Conrad from Los Angeles travelling to London on 10/1/2018
Issac Collet from Seattle travelling to London on 01/15/2018


In [58]:
import urllib.parse, json

search_terms = input("Search?\n")
traveller = input("Traveller?\n")

# Define the search parameters
searchParams = urllib.parse.urlencode({
    'search':'"{0}"'.format(search_terms),
    'searchMode':'All',
    'queryType':'full',
    '$count':'true',
    '$select':'url, name, address, city, post_code, birth_date, destination_city, destination_country, departure_date, return_date, policy_date',
    '$filter':"name eq '{0}'".format(traveller),
    'api-version':'22019-05-06-Preview'
})

try:
    result = azsearch_query(index="form-recognizer-index", params=searchParams)
    print('Hits:',result['@odata.count'])
    
    for value in result["value"]:
        print(value["name"], "from", value["city"], "travelling to", value["destination_city"], "on", value["departure_date"])
except Exception as e:
    print('Error:')
    print(e)

Search?
Las vegas
Traveller?
Chad Mouton
Hits: 1
Chad Mouton from Edinburgh travelling to Las Vegas on 11/18/2018
